*We recommend first reading this [article](https://txt.cohere.com/rag-chatbot-connector/) as it provides the context needed to understand this notebook.*

In [1]:
import cohere
import os
import uuid
from typing import List, Dict

co = cohere.Client("COHERE_API_KEY")

### Define and process documents

In [4]:
import requests

# Define the sources for the documents
# As an example, we'll use LLM University's Module 1: What are Large Language Models?
# https://docs.cohere.com/docs/intro-large-language-models

sources = [
    {
        "title": "Text Embeddings", 
        "url": "https://docs.cohere.com/docs/text-embeddings"},
    {
        "title": "Similarity Between Words and Sentences", 
        "url": "https://docs.cohere.com/docs/similarity-between-words-and-sentences"},
    {
        "title": "The Attention Mechanism", 
        "url": "https://docs.cohere.com/docs/the-attention-mechanism"},
    {
        "title": "Transformer Models", 
        "url": "https://docs.cohere.com/docs/transformer-models"}   
]

DEMO_CONNECTOR_API_KEY = "YOUR_CONNECTOR_API_KEY" # Replace with your connector API key

headers = {
    "Authorization": f"Bearer {DEMO_CONNECTOR_API_KEY}"
}

response = requests.post("YOUR_API_URL/process", # Replace with your API's URL
                         json={"sources": sources},
                         headers=headers)

print(response.json()["message"])

Documents processed successfully


### Chatbot component

In [3]:
class Chatbot:
    def __init__(self, connectors: List[str]):
        self.conversation_id = str(uuid.uuid4())
        self.connectors = [{"id": c} for c in connectors]


    def generate_response(self, message: str):
        response = co.chat(
                        message=message,
                        connectors=self.connectors,
                        conversation_id=self.conversation_id,
                        stream=True
                        )

        for event in response:
            yield event
        yield response

### App component

In [4]:
class App:
    def __init__(self, chatbot: Chatbot):
        self.chatbot = chatbot
        
    def run(self):
        while True:
            # Get the user message
            message = input("User: ")

            # Typing "quit" ends the conversation
            if message.lower() == "quit":
                print("Ending chat.")
                break
            else:
                print(f"User: {message}")

            # Get the chatbot response
            response = self.chatbot.generate_response(message)

            # Print the chatbot response
            print("Chatbot:")
            
            citations_flag = False
            
            for event in response:
                stream_type = type(event).__name__
                
                # Text
                if stream_type == "StreamTextGeneration":
                    print(event.text, end="")

                # Citations
                if stream_type == "StreamCitationGeneration":
                    if not citations_flag:
                        print("\n\nCITATIONS:")
                        citations_flag = True
                    print(event.citations[0])
                
                # Documents
                if citations_flag:
                    if stream_type == "StreamingChat":
                        print("\n\nDOCUMENTS:")
                        documents = [{'id': doc['id'],
                                      'text': doc['text'][:50] + '...',
                                      'title': doc['title'],
                                      'url': doc['url']} 
                                      for doc in event.documents]
                        for doc in documents:
                            print(doc)

            print(f"\n{'-'*100}\n")

### Run chatbot

In [10]:
# Define connectors
connectors = ["demo-conn-e5y5ps"]

# Create an instance of the Chatbot class by supplying the connectors
chatbot = Chatbot(connectors)

# Create an instance of the App class with the Chatbot instance
app = App(chatbot)

# Run the chatbot
app.run()

User: What are sentence embeddings
Chatbot:
Sentence embeddings are the building blocks of language models. They associate each sentence with a vector (list of numbers) in a way that similar sentences are assigned similar vectors. These vectors are composed of numbers and carry important properties of the sentence. The embeddings act as a form of translation between languages as well, as they provide a relatable vector for similar sentences in different languages.

CITATIONS:
{'start': 69, 'end': 124, 'text': 'associate each sentence with a vector (list of numbers)', 'document_ids': ['demo-conn-e5y5ps_0', 'demo-conn-e5y5ps_1', 'demo-conn-e5y5ps_2']}
{'start': 139, 'end': 186, 'text': 'similar sentences are assigned similar vectors.', 'document_ids': ['demo-conn-e5y5ps_0', 'demo-conn-e5y5ps_1']}
{'start': 235, 'end': 272, 'text': 'important properties of the sentence.', 'document_ids': ['demo-conn-e5y5ps_1', 'demo-conn-e5y5ps_2']}


DOCUMENTS:
{'id': 'demo-conn-e5y5ps_0', 'text': 'In th